In [91]:
from sklearn.externals import joblib 
from sklearn.neighbors import NearestNeighbors
from google.cloud import bigquery
import pandas as pd 
import numpy as np
from PIL import Image, ImageStat
import numpy as np
import math
import urllib.request

LOAD PRETRAINED MODEL AND HELPER FUNCTIONS

In [94]:
# load pretained model
knn_model = joblib.load("knn-model.jlib")  
data = pd.read_csv("indexed_books.csv")
indexed_books = data['0'].values

In [95]:
def show_color(colors):
    fig,ax = plt.subplots()
    currentAxis = plt.gca()
    x=0
    y=0
    width = 1/len(colors)
    for rgb in colors:
        colour = binascii.hexlify(bytearray(int(c) for c in rgb)).decode('ascii')
        colour = '#'+colour
        currentAxis.add_patch(Rectangle((x, y), width, 1, alpha=1, facecolor=colour))
        x=x+width

def dominant_colors(ar):
    NUM_CLUSTERS = 5
    codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)
    vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
    counts, bins = scipy.histogram(vecs, len(codes))    # count occurrences
    index_max = scipy.argmax(counts)                    # find most frequent
    peak = codes[index_max]
    return codes , peak

def image_colorfulness(image):
    R = np.array([x[0] for x in image])
    G = np.array([x[1] for x in image])
    B = np.array([x[2] for x in image])
    rg = np.absolute(R - G)
    yb = np.absolute(0.5 * (R + G) - B)
    (rbMean, rbStd) = (np.mean(rg), np.std(rg))
    (ybMean, ybStd) = (np.mean(yb), np.std(yb))
    stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
    meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
    return stdRoot + (0.3 * meanRoot)

def image_brightness(im):
   stat = ImageStat.Stat(im)
   r,g,b = stat.mean
   return math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

USE TEST IMAGE (REPLACE WITH WEBSITE UPLOADED IMAGE)

In [96]:
im = Image.open(urllib.request.urlopen("https://images-na.ssl-images-amazon.com/images/I/41vgassjrKL._SX329_BO1,204,203,200_.jpg"))
ar = np.asarray(im)
shape = ar.shape
ar = ar.reshape(scipy.product(shape[:2]), shape[2]).astype(float)

colorfullness = image_colorfulness(ar)
bright = image_brightness(im)
codes, peak = dominant_colors(ar)

testImage = [round(peak[0],2),round(peak[2],2),round(peak[2],2),round(bright,2),round(colorfullness,2)]
print(testImage)

[9.01, 190.12, 190.12, 103.72, 86.0]


APPLY MODEL TO IMAGE AND GENERATE SUGGESTIONS

In [97]:
d,i = knn_model.kneighbors([testImage])

In [101]:
ASIN = '"' + '","'.join(indexed_books[i[0]].tolist()) + '"'
client = bigquery.Client()

sql = """
SELECT *
FROM
    `eecs-e6893-book-cover.book_metadata.book_ratings`
WHERE
    ASIN in ({})
""".format(ASIN)
client.query(sql).to_dataframe()

,total_reviews,rating,ASIN
0,35.0,5.000000,0781776236
1,329.0,4.164557,0061789089


In [102]:
sql = """
SELECT *
FROM
    `eecs-e6893-book-cover.book_metadata.book_rankings`
WHERE
    ASIN in ({})
""".format(ASIN)
client.query(sql).to_dataframe()

,ranking,also_bought,ASIN
0,1168607,"1451193629,032342970X,032335517X,0838578535,14...",0781776236
1,253513,"0133416453,041587923X,0415657067,0872201589,08...",0415259215
2,38020,"0062383167,0205609996,0142181021,0470620749,01...",0061789089
